# Civil Engineering Machine Learning Project
## Damage evaluation of reinforced concrete beams with varying thickness using the acoustic emission technique


***
## 1. Data Extraction

In [62]:
"""
Created on Tue Jul 18 03:12:39 2017

@author: Aamir
"""

'\nCreated on Tue Jul 18 03:12:39 2017\n\n@author: Aamir\n'

### Import Libraries

In [63]:

import os
import scipy.io
import numpy as np
import pandas as pd
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.models import load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

### Import and select relevant data 

In [64]:
data = pd.read_excel("concrete-damage-data.xlsx")
data = data.iloc[:,[4, 5, 6, 7, 8, 9, 10, 17, 18, 19]]
data = data.dropna()
data.reset_index(drop=True,inplace=True)

### Check data frame structure 

In [65]:
data.shape

(82873, 10)

In [66]:
data.dtypes

CH              object
RISE            object
COUN            object
ENER            object
DURATION        object
AMP             object
A_FRQ           object
SIG_STRNGTH     object
ABS_ENERGY      object
DAMAGE         float64
dtype: object

In [67]:
data.head()

,CH,RISE,COUN,ENER,DURATION,AMP,A_FRQ,SIG_STRNGTH,ABS_ENERGY,DAMAGE
0,4,9,5,2,114,52,44,14372,260.846,0.0
1,3,1,1,0,1,45,1000,115.9,2.124,0.0
2,3,23,3,1,69,48,43,7241,107.876,0.0
3,4,1,2,1,79,46,25,7024,85.976,0.0
4,3,22,3,1,86,48,35,9022,133.646,0.0


***
## 2. Data Cleaning
### Check and fill missing values 

In [68]:
data.count()

CH             82873
RISE           82873
COUN           82873
ENER           82873
DURATION       82873
AMP            82873
A_FRQ          82873
SIG_STRNGTH    82873
ABS_ENERGY     82873
DAMAGE         82873
dtype: int64

No missing data. Incomplete entries have already been removed.

### Convert to numeric columns 

In [69]:
data['CH'] = data['CH'].astype(int)
data['RISE'] = data['RISE'].astype(int)
data['COUN'] = data['COUN'].astype(int)
data['ENER'] = data['ENER'].astype(int)
data['DURATION'] = data['DURATION'].astype(int)
data['AMP'] = data['AMP'].astype(int)
data['A_FRQ'] = data['A_FRQ'].astype(int)
data['SIG_STRNGTH'] = data['SIG_STRNGTH'].astype(float)
data['ABS_ENERGY'] = data['ABS_ENERGY'].astype(float)
data['DAMAGE'] = data['DAMAGE'].astype(int)

data.dtypes

CH               int32
RISE             int32
COUN             int32
ENER             int32
DURATION         int32
AMP              int32
A_FRQ            int32
SIG_STRNGTH    float64
ABS_ENERGY     float64
DAMAGE           int32
dtype: object

In [70]:
data.to_excel("Aamirdata.xlsx")

In [71]:
data.head()

,CH,RISE,COUN,ENER,DURATION,AMP,A_FRQ,SIG_STRNGTH,ABS_ENERGY,DAMAGE
0,4,9,5,2,114,52,44,14372.0,260.846,0
1,3,1,1,0,1,45,1000,115.9,2.124,0
2,3,23,3,1,69,48,43,7241.0,107.876,0
3,4,1,2,1,79,46,25,7024.0,85.976,0
4,3,22,3,1,86,48,35,9022.0,133.646,0


### Normalize Data

In [72]:
mn = pd.DataFrame.mean(data)
vr = pd.DataFrame.var(data)

In [73]:
data.RISE = (data.RISE - mn.RISE) / vr.RISE
data.COUN = (data.COUN - mn.COUN) / vr.COUN
data.ENER = (data.ENER - mn.ENER) / vr.ENER
data.DURATION = (data.DURATION - mn.DURATION) / vr.DURATION
data.AMP = (data.AMP - mn.AMP) / vr.AMP
data.A_FRQ = (data.A_FRQ - mn.A_FRQ) / vr.A_FRQ
data.SIG_STRNGTH = (data.SIG_STRNGTH - mn.SIG_STRNGTH) / vr.SIG_STRNGTH
data.ABS_ENERGY = (data.ABS_ENERGY - mn.ABS_ENERGY) / vr.ABS_ENERGY

In [74]:
data.head()

,CH,RISE,COUN,ENER,DURATION,AMP,A_FRQ,SIG_STRNGTH,ABS_ENERGY,DAMAGE
0,4,-0.000108,-0.000432,-0.000407,-0.000019,-0.022753,-0.000936,-6.540712e-08,-8.322200e-08,0
1,3,-0.000109,-0.000480,-0.000426,-0.000020,-0.143455,0.014636,-6.886812e-08,-8.384964e-08,0
2,3,-0.000106,-0.000456,-0.000417,-0.000020,-0.091726,-0.000952,-6.713834e-08,-8.359309e-08,0
3,4,-0.000109,-0.000468,-0.000417,-0.000020,-0.126212,-0.001245,-6.719102e-08,-8.364622e-08,0
4,3,-0.000106,-0.000456,-0.000417,-0.000020,-0.091726,-0.001082,-6.670596e-08,-8.353058e-08,0


In [75]:
predictors = np.array(data)
target = np.eye(4)[data['DAMAGE'].astype(int)]
n_cols = predictors.shape[1]

***
## 3. Train and Evaluate the Model
### Set target, features to train model 

### Specify the model

In [77]:
# Specify the model
model = Sequential()
model.add(Dense(10, activation='relu', input_shape = (n_cols,)))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(4, activation='softmax'))

early_stopping_monitor = EarlyStopping(patience=2)

# Compile the model
sgd = SGD(lr=0.1)#, decay=1e-6, momentum=1.9)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(predictors, target, validation_split=0.3, batch_size=64, epochs=100, callbacks=[early_stopping_monitor])


Train on 58011 samples, validate on 24862 samples
Epoch 1/100
58011/58011 [==============================] - 6s - loss: 0.6345 - acc: 0.7644 - val_loss: 0.1570 - val_acc: 1.0000
Epoch 2/100
58011/58011 [==============================] - 5s - loss: 0.0065 - acc: 0.9998 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 3/100
58011/58011 [==============================] - 5s - loss: 0.0027 - acc: 0.9997 - val_loss: 9.5068e-04 - val_acc: 1.0000
Epoch 4/100
58011/58011 [==============================] - 5s - loss: 0.0023 - acc: 0.9998 - val_loss: 5.7264e-04 - val_acc: 1.0000
Epoch 5/100
58011/58011 [==============================] - 5s - loss: 0.0022 - acc: 0.9998 - val_loss: 4.1161e-04 - val_acc: 1.0000
Epoch 6/100
58011/58011 [==============================] - 5s - loss: 0.0021 - acc: 0.9998 - val_loss: 3.0965e-04 - val_acc: 1.0000
Epoch 7/100
58011/58011 [==============================] - 5s - loss: 0.0020 - acc: 0.9998 - val_loss: 2.5516e-04 - val_acc: 1.0000
Epoch 8/100
58011/58011 [=========

58011/58011 [==============================] - 18s - loss: 6.9437e-06 - acc: 1.0000 - val_loss: 1.7280e-05 - val_acc: 1.0000
Epoch 61/100
58011/58011 [==============================] - 20s - loss: 6.7862e-06 - acc: 1.0000 - val_loss: 1.6942e-05 - val_acc: 1.0000
Epoch 62/100
58011/58011 [==============================] - 19s - loss: 6.6332e-06 - acc: 1.0000 - val_loss: 1.6627e-05 - val_acc: 1.0000
Epoch 63/100
58011/58011 [==============================] - 9s - loss: 6.4896e-06 - acc: 1.0000 - val_loss: 1.6280e-05 - val_acc: 1.0000
Epoch 64/100
58011/58011 [==============================] - 9s - loss: 6.3500e-06 - acc: 1.0000 - val_loss: 1.5921e-05 - val_acc: 1.0000
Epoch 65/100
58011/58011 [==============================] - 9s - loss: 6.2203e-06 - acc: 1.0000 - val_loss: 1.5633e-05 - val_acc: 1.0000
Epoch 66/100
58011/58011 [==============================] - 7s - loss: 6.0914e-06 - acc: 1.0000 - val_loss: 1.5338e-05 - val_acc: 1.0000
Epoch 67/100
58011/58011 [=========================

### Performance evaluation 

In [ ]:
# Look at the importance and score of the included features


***
## 5. Predict the Test set

### Select test model features 

### Make predictions 

***
## 6. Save Results
### Create the Solution 

In [ ]:
my_solution.index.name

### Save to CSV file 

In [ ]:
my_solution.to_csv("my_solution_six.csv")

***
***
# Rough Work 

In [ ]:
%matplotlib inline